This notebook documents **our Julia notebooks and how to run on the Google compute cloud.**

<h1 id="tocheading">TABLE OF CONTENTS</h1>
<div id="toc"></div>

**Updates to the table of contents are periodic, but run the cell below to first start or force an update.**

In [3]:
macro javascript_str(s) display("text/javascript", s); end

javascript"""
$.getScript('make_table_of_contents.js')
"""


# Overview of .ipynb notebooks and their associated .jl files

The notebooks in this directory contain code to run and differentiate ProAnti models. Each notebook contains some core source code, that gets auto-extracted into a Julia .jl file, plus documentation and examples that are for illustration and are not extracted into the .jl file.

The notebooks are:

**[Scraper.ipynb](Scraper.ipynb)** : defines and describes how to auto-scrape notebooks into .jl files, extracted into <span style="color:red">scraper.jl</span>.

**[General Utilities.ipynb](General Utilities.ipynb)**  : various utilities, extracted into <span style="color:red">general_utils.jl</span>.

**[Gradient Utilities.ipynb](Gradient Utilities.ipynb)**  : defining `vgh()` and `keyword_vgh()` (for computing value, gradient, hessian all together) as well as `make_dict()` and others.  Extracted into <span style="color:red">gradient_utils.jl</span>.  

**[Constrained Parabolic Minimization.ipynb](Constrained Parabolic Minimization.ipynb)** : finding the minimum of a parabolic surface, constrained to lie within a given radius from the origin. Used by the next notebook, and extracted into <span style="color:red">constrained_parabolic_minimization.jl</span>  Including it also includes `gradient_utils.jl`  

**[Optimization Utilities.ipynb](Optimization Utilities.ipynb)** : like the title says, for optimizing functions. Includes utilities for the soft `tanh()` method for bounding parameter walls, and includes `bbox_hessian_keyword_minimization()`. Extracted into <span style="color:red">optimization_utils.jl</span> Including it also includes `constrained_parabolic_minimization.jl` and `general_utils.jl`.

**[Rate Networks.ipynb](Rate Networks.ipynb)** : running and differentiating arbitrary rate-based neural networks, extracted into <span style="color:red">rate_networks.jl</span>. Including `rate_networks.jl` will also include all three of `optimization_utils.jl`.

**[ProAnti.ipynb](ProAnti.ipynb)** : Uses all of the above to run and optimize ProAnti models. The model definition, trial running, and the cost function we started with are extracted into <span style="color:red">pro_anti.jl</span>. An example of code for running an optimization farm is extracted into <span style="color:red">farming.jl</span> Including it will also include `rate_networks.jl`.

# Running on the Google compute cloud

You'll want to both install [google compute command line tools](https://cloud.google.com/sdk/) and find your way to the [compute Console web site](https://console.cloud.google.com/compute/instances?project=autoreminder-1219).  Our project is called "Autoreminder".

Going from the Console web site, you can go to the "VM Instances" and there see the VMs we have configured so far. Their disk drive is persistent, meaning that whatever is on it stays on it even when the machine is turned off. We get charged a very small amount per GB of hard disk, regardless of whether VMs are on or off. This amount is so small that I don't think about it. For CPU cycles, we get charged by the core by the minute when a VM is turned on.  A 64-core VM that is turned on runs at about $3/hour (again, regardless of CPU usage).

Our quota is up to 2048 CPUs, located in Google's "us-east1" zones.

## Making a new instance and initializing it (i.e., adding julia, git, etc.) 

### Making the instance

To make a new instance, do it from the Console web site. Choose one the the "us-east1" zones  (for example, "us-east1-c", which is where we have a high CPU quota, and choose the default Debian 9 Linux operating system. Most of the machines made so far are 64-core machines with 10 GB hard drive (we don't need a lot of disk space).

### Logging into the instance

Lets say you made an instance (i.e., a VM) and called it "my_instance" and have turned it on (clicked "start" for that instance from the VM instances console). If you have installed the gcloud command-line tool, you can ssh into your new instance using

    gcloud compute ssh my_google_username@my_instance
 
The gcloud tool takes care of figuring out the correct IP address, etc. And I _believe_ that the first time you try this it'll guide you through setting up the ssh keys, but I'm not totally sure about that. Further googling on how to intialize your ssh keys might be needed to allow you to log in without having to type a password every time.

You can ssh in and then treat the VM as a regular machine. Or, you can run a command remotely, e.g.

    gcloud compute ssh --command "echo 'all the way from Google'" my_google_username@my_instance
    
Or, you can scp files,  (don't forget the colon at the end to indicate remote machine!) e.g. copying into your home directory

    gcloud compute scp package_installs.sh julia_start.jl grow_ForwardDiffChunks.sh my_google_username@my_instance:

### Initializing the instance with Julia, our repository, and so on.

First, edit the file `package_installs.sh` and in the git clone line put in **your** github username and password. Then run the following two commands on your local machine:

    gcloud compute scp package_installs.sh julia_start.jl grow_ForwardDiffChunks.sh my_google_username@my_instance:
    gcloud compute ssh --command "./package_installs.sh" my_google_username@my_instance
    
That will install Emacs, Julia, Git, and will then start up Julia and get the various packages that we use (ForwardDiff, HDF5, MAT, etc) and will then install our git repository in the directory `superior_colliculus_mutual_inhibition`

### Making, initializing, and stopping a new 64-core instance all from the command-line

Here's a summary for making a new 64-core instance, where `my_instance` should be the name of the new VM

    gcloud compute instances create my_instance --image-family debian-9 --image-project debian-cloud --machine-type n1-standard-64 --zone us-east1-c
    
(or a different us-east1- zone if c is busy)

Then edit package_installs.sh so that in the git clone line you put in **your** github username and password. Then

    gcloud compute scp package_installs.sh julia_start.jl my_google_username@my_instance:
    gcloud compute ssh --command "./package_installs.sh" my_google_username@my_instance
    gcloud compute instances stop my_instance
    
### UN-setting your default gcloud zone to work with multiple zones

If you don't have a particular gcloud zone set as your default, then gcloud will try to figure out which zone you mean based on the machine name, which is convenient. If you did set a default zone and want to unset it, Marino recommends

    gcloud config unset compute/zone
    gcloud config unset compute/region
    

## Starting multiple identical Julia process

Within our repository, there's a little helper shell script for starting a bunch of identical Julia processes running a given Julia file. It's called `start_farm.sh`.

It turns out that if you start a process in the background, and that process writes some text to the terminal, and you have closed the terminal, the process will die. And it also turns out that if you try to start too mahy Julia processes simultaneously, some of them crash (I don't know why, I wonder whether it is because of too many of them precompiling stuff simultaneously and overwriting each other). `start_farm.sh` deals with both of those things by (a) piping each process' output into separate text files, numbered as `julia_outs_n`; and (b) it sleeps for 3 seconds between cycles of starting up a process. 


Run it on its own to get a little usage statement.

    unix> ./start_farm.sh

    Sorry, need two arguments, the name of the julia file to farm, and the number of farms to start
   
**Example:**

    unix> ./start_farm.sh  farm.jl  60
    
will start 60 separate Julia processes, each running  `julia farm.jl > julia_output_n` where `n` is different for eac process.


## Pulling results from the cloud

Another little helper sheel script in the repository is for ssh-ing into a machine; it then does git pull to make sure the VM's copy of the repository is synchronized; then adds the VM's new farm animals to the git reporitory, commits them, and pushes them to the github server; and then, back in the local machine, does `git pull` to bring them down from the github repository. **For it to work for you, you probably need to edit it to have your google username instead of "carlosbrody".**

Run it on its own to get a little usage statement and an example:

    unix> ./pull_results.sh
    
    Sorry, need two arguments, the name of the gcloud machine and the letter of the farm being created there
    For example: ./pull_results.sh proanti003 D
    
The farm animals (i.e. files) it looks for should be of the form `FarmFields/farm_{MYLETTER}_` where `{MYLETTER}` is the second argument to `pull_results.sh`, in the example above the letter `D`.

Other than the usage statement, the action part of `pull_results.sh` is simply:

    gcloud compute ssh --zone="us-east1-c" --command="cd superior_colliculus_mutual_inhibition/; git pull; git add FarmFields/farm_$argv[2]_*; git commit -m "'"'"The latest from farm $argv[2]"'"'"  ; git push origin master" carlosbrody@$argv[1] 

    git pull
    
Have fun!
